In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
# Set the root directory of the dataset
root_dir = '/content/drive/MyDrive/Data'

# Create a dictionary to map the label names to integers
label_map = {'Sofa': 0, 'Bed': 1, 'Chair': 2}

In [ ]:
# Create a list of all the image file paths and their corresponding labels
data = []
for label in label_map.keys():
    path = os.path.join(root_dir, label)
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            file_path = os.path.join(path, filename)
            label_id = label_map[label]
            data.append((file_path, label_id))

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Create the training and testing directories and copy the images
train_dir = 'train/'
test_dir = 'test/'

for label_id in label_map.values():
    os.makedirs(os.path.join(train_dir, str(label_id)), exist_ok=True)
    os.makedirs(os.path.join(test_dir, str(label_id)), exist_ok=True)

for file_path, label_id in train_data:
    filename = os.path.basename(file_path)
    shutil.copy(file_path, os.path.join(train_dir, str(label_id), filename))

for file_path, label_id in test_data:
    filename = os.path.basename(file_path)
    shutil.copy(file_path, os.path.join(test_dir, str(label_id), filename))

In [5]:
# Define transforms for training and testing datasets
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
# Load the data
train_data = datasets.ImageFolder('/content/train', transform=train_transforms)
test_data = datasets.ImageFolder('/content/test', transform=test_transforms)
from torch.utils.data import DataLoader
# # Define the data loaders for the training and testing sets
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)


In [ ]:
# Define the model architecture
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_data.classes))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Train the model
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [33]:
# Set the model to train mode
model.train()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(num_epochs):
    # Training
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            accuracy += (logits.argmax(1) == labels).float().sum()
        accuracy /= len(test_data)
    print(f"Epoch {epoch+1}, Validation Accuracy: {accuracy.item()*100}%")

# Save the trained model parameters
torch.save(model.state_dict(), 'model.pth')

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

# Load the saved model checkpoint
checkpoint = torch.load('model.pth')

In [ ]:
# Load the model architecture
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

In [ ]:
# Load the model state dictionary
model.load_state_dict(torch.load('model.pth'))
#model.load_state_dict(checkpoint['state_dict'])

# Set the model to evaluation mode
model.eval()

In [ ]:
# Define the image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Load the image
image = Image.open('/content/3 SEATER SOFA OG.JPG')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

In [ ]:
# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()
    
print(predicted_class)

In [36]:
# Define the label mapping
label_map = {0:"Sofa", 1:"Bed", 2:"Chair"}

In [ ]:
# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)

In [ ]:
# Load the image
image = Image.open('/content/chair2.png')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()
    
print(predicted_class)
# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)

In [ ]:
# Load the image
image = Image.open('/content/bed.jpg')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()  
print(predicted_class)

# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)